# Title: Potato Crop Disease Classification

#### Group Member Names : Sukhjeet Kaur (200575307)



### INTRODUCTION:
*********************************************************************************************************************
#### AIM : The aim of this project is to replicate and extend the research presented in the paper "Soybean disease identification using original field images and transfer learning with convolutional neural networks" by adapting the methodology for a different crop disease dataset. Specifically, we have used a potato dataset to evaluate the performance of the transfer learning approach in identifying potato crop diseases.

*********************************************************************************************************************
#### Github Repo: The code and resources for the project are available in the GitHub repository: https://github.com/nzb0054/RoboCrop-CNN-WebApp

*********************************************************************************************************************
#### DESCRIPTION OF PAPER: The paper "Soybean disease identification using original field images and transfer learning with convolutional neural networks" presents a methodology for identifying soybean diseases using convolutional neural networks (CNNs) with transfer learning. The authors employed pre-trained CNN models to leverage learned features from large-scale datasets and fine-tuned them for soybean disease classification.

*********************************************************************************************************************
#### PROBLEM STATEMENT : Identifying crop diseases accurately is crucial for effective crop management and yield optimization. Traditional methods of disease identification are labor-intensive and often lack precision. The problem is to enhance disease identification accuracy using image-based methods, specifically focusing on potato crop diseases in this study.

*********************************************************************************************************************
#### CONTEXT OF THE PROBLEM: Potato crops are susceptible to various diseases that can significantly impact yield and quality. Early and accurate disease detection can help in timely intervention, reducing the potential loss. Utilizing image-based classification techniques with transfer learning provides an opportunity to improve the efficiency and accuracy of disease identification.



##Clone the GitHub Repository:

In [ ]:
!git clone https://github.com/nzb0054/RoboCrop-CNN-WebApp.git
%cd RoboCrop-CNN-WebApp

Cloning into 'RoboCrop-CNN-WebApp'...
remote: Enumerating objects: 44, done.
remote: Counting objects: 100% (44/44), done.
remote: Compressing objects: 100% (43/43), done.
remote: Total 44 (delta 21), reused 3 (delta 1), pack-reused 0
Receiving objects: 100% (44/44), 11.25 MiB | 10.24 MiB/s, done.
Resolving deltas: 100% (21/21), done.
/content/RoboCrop-CNN-WebApp


##Image size and batch size initialization

In [ ]:
img_height, img_width = (150, 150)
batch_size = 32

#Data Loading and Augmentation

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
    width_shift_range=0.2,
    height_shift_range=0.2,
    fill_mode="nearest",
    brightness_range=[0.9, 1.1],
    rotation_range=30,
    vertical_flip=True,
    horizontal_flip=True,
    validation_split=0.05,
    rescale=1./255
)

train_generator = train_datagen.flow_from_directory(
    '/content/drive/MyDrive/PLD_3_Classes_256 2/Training',
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    subset='training'
)

valid_generator = train_datagen.flow_from_directory(
    '/content/drive/MyDrive/PLD_3_Classes_256 2/Validation',
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation'
)

test_generator = train_datagen.flow_from_directory(
    '/content/drive/MyDrive/PLD_3_Classes_256 2/Testing',
    target_size=(img_height, img_width),
    batch_size=1,
    class_mode='categorical',
    subset='validation'
)

Found 627 images belonging to 3 classes.
Found 20 images belonging to 3 classes.
Found 20 images belonging to 3 classes.


#Load the Base Model and Build the Model:

In [ ]:
from tensorflow.keras.applications import DenseNet201
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.models import Model

base_model = DenseNet201(include_top=False, weights='imagenet')
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.2)(x)
x = Dense(64, activation='relu')(x)
x = Dropout(0.3)(x)
predictions = Dense(train_generator.num_classes, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=predictions)

for layer in base_model.layers:
    layer.trainable = False

#Compile the Model

In [ ]:
from tensorflow.keras.optimizers import SGD

model.compile(optimizer=SGD(learning_rate=0.0001, momentum=0.9),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

#Train the Model for 30 epochs

In [ ]:
from tensorflow.keras.callbacks import CSVLogger

log_csv = CSVLogger('my_logs_robocrop.csv', separator=',', append=False)
callbacks_list = [log_csv]

model.fit(train_generator,
          epochs=30,
          validation_data=valid_generator,
          callbacks=callbacks_list,
          verbose=1)

Epoch 1/30


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


20/20 ━━━━━━━━━━━━━━━━━━━━ 98s 3s/step - accuracy: 0.7527 - loss: 0.6597 - val_accuracy: 0.4000 - val_loss: 1.5106
Epoch 2/30
20/20 ━━━━━━━━━━━━━━━━━━━━ 7s 229ms/step - accuracy: 0.9939 - loss: 0.2169 - val_accuracy: 0.4000 - val_loss: 2.1290
Epoch 3/30
20/20 ━━━━━━━━━━━━━━━━━━━━ 8s 254ms/step - accuracy: 0.9943 - loss: 0.0923 - val_accuracy: 0.4000 - val_loss: 2.6008
Epoch 4/30
20/20 ━━━━━━━━━━━━━━━━━━━━ 7s 241ms/step - accuracy: 0.9982 - loss: 0.0662 - val_accuracy: 0.4000 - val_loss: 2.8003
Epoch 5/30
20/20 ━━━━━━━━━━━━━━━━━━━━ 10s 238ms/step - accuracy: 1.0000 - loss: 0.0459 - val_accuracy: 0.4000 - val_loss: 2.9249
Epoch 6/30
20/20 ━━━━━━━━━━━━━━━━━━━━ 8s 300ms/step - accuracy: 1.0000 - loss: 0.0395 - val_accuracy: 0.4000 - val_loss: 3.1423
Epoch 7/30
20/20 ━━━━━━━━━━━━━━━━━━━━ 9s 248ms/step - accuracy: 1.0000 - loss: 0.0317 - val_accuracy: 0.4000 - val_loss: 3.4201
Epoch 8/30
20/20 ━━━━━━━━━━━━━━━━━━━━ 10s 229ms/step - accuracy: 1.0000 - loss: 0.0288 - val_accuracy: 0.4000 - val_

#Evaluate the Model

In [ ]:
test_loss, test_acc = model.evaluate(test_generator, verbose=2)
print('\nTest Accuracy:', test_acc)

20/20 - 18s - 882ms/step - accuracy: 0.4000 - loss: 4.9369

Test Accuracy: 0.4000000059604645


#### SOLUTION:

To address the problem, we adapted the methodology from the original soybean disease identification paper by **substituting** the dataset with potato images and adjusting the image size to **150x150 pixels**.

We utilized the **DenseNet201** model with all base layers set to non-trainable, trained it for **30 epochs**, and tailored the model for potato disease classification.

These modifications allowed us to effectively apply transfer learning techniques to a new crop context, demonstrating the model's versatility and efficacy in identifying potato diseases.